### Run the Weather-Routing-Tool with the given template. 

In [ ]:
%run load_env.py
%run delete_Images_WRT.py
%run cli.py -f config.template.json

### Display the calculated route.

In [ ]:
%run display_geojson.py

### You can go through the routing steps by moving the slider.

In [ ]:
%run routingstep_slider.py

### You can start youre own routing by setting the two map markers (startpoint:green, endpoint:red). 
### Press 'start routing' to calculate the route.
### If you want to set a new route press 'new route'.

In [ ]:
%run interactive_WRT.py